In [4]:
import pandas as pd
from datetime import datetime

In [5]:
xls = pd.ExcelFile('Annotation_Sheet(P108&P102&P113 - Edit).xlsx')

dfs = []
for sheet_name in xls.sheet_names:
    df = pd.read_excel('Annotation_Sheet(P108&P102&P113 - Edit).xlsx', sheetname=sheet_name)
    dfs.append(df)
    
annotation = pd.concat(dfs, join='inner')

In [6]:
#Write Discrepancies to CSV here? 

In [7]:
#Adding the Start time and end time of the clips as new names with date

annotation = annotation[annotation["Date"].notnull()]
annotation = annotation[annotation["StartClock"].notnull()]
annotation = annotation[annotation["EndClock"].notnull()]
#Because need all 3 {Date,StartClock, Endclock} to be not null

annotation['DateStartTimeClock'] = pd.to_datetime(annotation['Date'].apply(str)+' '+annotation['StartClock'].apply(str))
annotation['DateEndTimeClock'] = pd.to_datetime(annotation['Date'].apply(str)+' '+annotation['EndClock'].apply(str))
print(annotation.shape)
print(annotation['DateStartTimeClock'].head())
print(annotation.dtypes)

(4294, 17)
0   2018-01-18 16:07:40
1   2018-01-18 16:11:11
2   2018-01-18 16:11:17
3   2018-01-18 16:11:20
4   2018-01-18 16:11:23
Name: DateStartTimeClock, dtype: datetime64[ns]
ParticipantID                         float64
Accel_Waist                            object
Accel_Wrist                            object
Date                           datetime64[ns]
Sequence                              float64
VideoName                              object
VideoStartClock_camera                 object
VideoStartClock_timeanddate            object
StartRunTime                           object
EndRunTime                             object
StartClock                             object
EndClock                               object
ActivityLevel1                        float64
ActivityLevel2                        float64
ActivityLevel3                        float64
DateStartTimeClock             datetime64[ns]
DateEndTimeClock               datetime64[ns]
dtype: object


In [8]:
def fetchSignal(file_, df):
    print(df.shape)
    participant = file_.split("_")[0][-3:]
    print(participant)
    #fetch the single annoataion of the particpant
    participantannotation = annotation.loc[annotation['ParticipantID'] == float(participant)]
    print(participantannotation.shape)
    print(annotation.shape)
    tempframe = pd.DataFrame()
    # all the rows in the annotation
    for index, row in participantannotation.iterrows():
        signalrange = (df['Timestamp'] >= row['DateStartTimeClock']) & (df['Timestamp'] < row['DateEndTimeClock'])
        signalframe = df.loc[signalrange]
        signalframe['activity1'] = row['ActivityLevel1']
        signalframe['activity2'] = row['ActivityLevel2']
        signalframe['activity3'] = row['ActivityLevel3']
        signalframe['Accel_Waist'] = row['Accel_Waist']
        signalframe['Accel_Wrist'] = row['Accel_Wrist']
        signalframe['Sequence'] = row['Sequence']
        signalframe['ParticipantID'] = row['ParticipantID']
        tempframe = tempframe.append(signalframe)
    df = tempframe
    print(df.shape)
    print(df.head)
    return df

In [9]:
#Read all csv files in the folder and ignore the first 10 lines(The first 10 lines are explanation that is not neccesarry)
import glob
import pandas as pd
allFiles = glob.glob("../../ActivityRecognitionDataCSV/*waist*.csv")
list_ = []
#print(allFiles)
#file = 101.RAW.CSV
#df = df of 101 data  of gt3x
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,skiprows=10) #skiprows = ignore 10 first lines
    print("Processing file ", file_)
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) #convert string into datetime format
    df3 = fetchSignal(file_,df)  ##??
    list_.append(df3)
    print("Finished Processing file ", file_)
dataframe = pd.concat(list_)

#print("Column headings:")
#print(dataframe.columns)
#print(dataframe.dtypes)
#print(dataframe.shape)
print(dataframe.head())


Processing file  ../../ActivityRecognitionDataCSV/114_waistRAW.csv
(729000, 4)
114
(164, 17)
(4294, 17)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

(16020, 11)
<bound method NDFrame.head of                      Timestamp  Accelerometer X  Accelerometer Y  \
549000 2018-02-19 15:20:00.000            0.879           -0.242   
549001 2018-02-19 15:20:00.034            0.879           -0.242   
549002 2018-02-19 15:20:00.067            0.887           -0.250   
549003 2018-02-19 15:20:00.100            0.879           -0.246   
549004 2018-02-19 15:20:00.134            0.879           -0.250   
549005 2018-02-19 15:20:00.167            0.879           -0.246   
549006 2018-02-19 15:20:00.200            0.871           -0.250   
549007 2018-02-19 15:20:00.234            0.879           -0.254   
549008 2018-02-19 15:20:00.267            0.883           -0.254   
549009 2018-02-19 15:20:00.300            0.879           -0.250   
549010 2018-02-19 15:20:00.334            0.879           -0.246   
549011 2018-02-19 15:20:00.367            0.887           -0.254   
549012 2018-02-19 15:20:00.400            0.883           -0.250   
549013

Processing file  ../../ActivityRecognitionDataCSV/122_waistRAW.csv
(292296, 4)
122
(160, 17)
(4294, 17)
(30180, 11)
<bound method NDFrame.head of                     Timestamp  Accelerometer X  Accelerometer Y  \
18330 2018-03-29 11:25:11.000            0.125            0.051   
18331 2018-03-29 11:25:11.034            0.090            0.012   
18332 2018-03-29 11:25:11.067            0.090            0.020   
18333 2018-03-29 11:25:11.100            0.098            0.020   
18334 2018-03-29 11:25:11.134            0.094            0.023   
18335 2018-03-29 11:25:11.167            0.094            0.020   
18336 2018-03-29 11:25:11.200            0.098            0.020   
18337 2018-03-29 11:25:11.234            0.094            0.016   
18338 2018-03-29 11:25:11.267            0.098            0.023   
18339 2018-03-29 11:25:11.300            0.090            0.027   
18340 2018-03-29 11:25:11.334            0.094            0.020   
18341 2018-03-29 11:25:11.367            0.094    

Processing file  ../../ActivityRecognitionDataCSV/115_waistRAW.csv
(594000, 4)
115
(153, 17)
(4294, 17)
(28320, 11)
<bound method NDFrame.head of                     Timestamp  Accelerometer X  Accelerometer Y  \
46500 2018-03-15 12:55:50.000           -0.090            1.000   
46501 2018-03-15 12:55:50.034           -0.098            1.000   
46502 2018-03-15 12:55:50.067           -0.098            0.996   
46503 2018-03-15 12:55:50.100           -0.094            0.992   
46504 2018-03-15 12:55:50.134           -0.086            0.988   
46505 2018-03-15 12:55:50.167           -0.102            0.992   
46506 2018-03-15 12:55:50.200           -0.105            0.996   
46507 2018-03-15 12:55:50.234           -0.098            0.992   
46508 2018-03-15 12:55:50.267           -0.094            0.988   
46509 2018-03-15 12:55:50.300           -0.094            0.992   
46510 2018-03-15 12:55:50.334           -0.098            0.996   
46511 2018-03-15 12:55:50.367           -0.102    

...

Processing file  ../../ActivityRecognitionDataCSV/106_waistRAW.csv
(432000, 4)
106
(254, 17)
(4294, 17)
(28890, 11)
<bound method NDFrame.head of                      Timestamp  Accelerometer X  Accelerometer Y  \
162450 2018-01-29 15:00:15.000            0.617            0.844   
162451 2018-01-29 15:00:15.034            0.586            0.914   
162452 2018-01-29 15:00:15.067            0.500            0.781   
162453 2018-01-29 15:00:15.100            0.578            0.891   
162454 2018-01-29 15:00:15.134            0.520            0.879   
162455 2018-01-29 15:00:15.167            0.332            0.941   
162456 2018-01-29 15:00:15.200            0.410            0.805   
162457 2018-01-29 15:00:15.234            0.496            0.910   
162458 2018-01-29 15:00:15.267            0.836            0.652   
162459 2018-01-29 15:00:15.300            0.711            1.070   
162460 2018-01-29 15:00:15.334            0.672            1.074   
162461 2018-01-29 15:00:15.367        

Processing file  ../../ActivityRecognitionDataCSV/124_waistRAW.csv
(388060, 4)
124
(78, 17)
(4294, 17)
(15060, 11)
<bound method NDFrame.head of                      Timestamp  Accelerometer X  Accelerometer Y  \
90690  2018-03-30 10:35:23.000            0.035            0.059   
90691  2018-03-30 10:35:23.034            0.027            0.051   
90692  2018-03-30 10:35:23.067            0.031            0.047   
90693  2018-03-30 10:35:23.100            0.031            0.051   
90694  2018-03-30 10:35:23.134            0.027            0.047   
90695  2018-03-30 10:35:23.167            0.031            0.047   
90696  2018-03-30 10:35:23.200            0.027            0.055   
90697  2018-03-30 10:35:23.234            0.035            0.055   
90698  2018-03-30 10:35:23.267            0.035            0.047   
90699  2018-03-30 10:35:23.300            0.031            0.051   
90700  2018-03-30 10:35:23.334            0.035            0.055   
90701  2018-03-30 10:35:23.367         

Processing file  ../../ActivityRecognitionDataCSV/119_waistRAW.csv
(378000, 4)
119
(93, 17)
(4294, 17)
(15810, 11)
<bound method NDFrame.head of                     Timestamp  Accelerometer X  Accelerometer Y  \
25950 2018-03-20 09:59:25.000            0.699           -0.629   
25951 2018-03-20 09:59:25.034            0.688           -0.625   
25952 2018-03-20 09:59:25.067            0.703           -0.652   
25953 2018-03-20 09:59:25.100            0.711           -0.664   
25954 2018-03-20 09:59:25.134            0.734           -0.734   
25955 2018-03-20 09:59:25.167            0.777           -0.832   
25956 2018-03-20 09:59:25.200            0.902           -0.992   
25957 2018-03-20 09:59:25.234            0.969           -1.281   
25958 2018-03-20 09:59:25.267            1.023           -1.234   
25959 2018-03-20 09:59:25.300            0.836           -0.973   
25960 2018-03-20 09:59:25.334            0.441           -0.434   
25961 2018-03-20 09:59:25.367            0.066     

Processing file  ../../ActivityRecognitionDataCSV/116_waistRAW.csv
(729000, 4)
116
(175, 17)
(4294, 17)
(27270, 11)
<bound method NDFrame.head of                     Timestamp  Accelerometer X  Accelerometer Y  \
58500 2018-03-16 10:47:30.000            1.547           -4.656   
58501 2018-03-16 10:47:30.034            0.813           -2.949   
58502 2018-03-16 10:47:30.067            0.188           -1.230   
58503 2018-03-16 10:47:30.100           -0.043            0.715   
58504 2018-03-16 10:47:30.134           -0.223            1.945   
58505 2018-03-16 10:47:30.167            0.664            0.332   
58506 2018-03-16 10:47:30.200            1.559           -4.938   
58507 2018-03-16 10:47:30.234            0.637           -2.359   
58508 2018-03-16 10:47:30.267            0.219           -1.063   
58509 2018-03-16 10:47:30.300            0.070            0.418   
58510 2018-03-16 10:47:30.334           -0.207            1.816   
58511 2018-03-16 10:47:30.367            0.547    

Processing file  ../../ActivityRecognitionDataCSV/121_waistRAW.csv
(297000, 4)
121
(168, 17)
(4294, 17)
(33480, 11)
<bound method NDFrame.head of                     Timestamp  Accelerometer X  Accelerometer Y  \
38700 2018-03-26 10:06:30.000            0.270           -0.223   
38701 2018-03-26 10:06:30.034            0.234           -0.168   
38702 2018-03-26 10:06:30.067            0.148           -0.051   
38703 2018-03-26 10:06:30.100           -0.066            0.059   
38704 2018-03-26 10:06:30.134           -0.152            0.180   
38705 2018-03-26 10:06:30.167           -0.059            0.328   
38706 2018-03-26 10:06:30.200            0.285            0.613   
38707 2018-03-26 10:06:30.234            0.180            0.313   
38708 2018-03-26 10:06:30.267            0.180            0.074   
38709 2018-03-26 10:06:30.300            0.012            0.020   
38710 2018-03-26 10:06:30.334           -0.211           -0.320   
38711 2018-03-26 10:06:30.367           -0.516    

### Save the big cleaned dataframe in the pickle

In [10]:
import pickle
# ?? how to activate pickle
pickling_on = open("wholedatasignal.pickle","wb")
pickle.dump(dataframe, pickling_on)

In [11]:
dataframe = dataframe.dropna() #drops rows or columns that have missing value
dataframe = dataframe.reset_index(drop=True)

In [12]:
dataframe.shape


(542070, 11)

In [17]:
import itertools
import numpy as np
from io import StringIO
import math
import scipy.stats.stats as stats
import matplotlib.pyplot as plt 
import pandas as pd

### Splitting two second window for activity level 1

In [ ]:
data = None 
data = dataframe #data.head to see data (combined data of annotation and raw data gt3x)
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''  # activity 
subvalue = '' # participant number
for ind, ro in data.iterrows():
    # to seperate the clips into 2 second clips(j<60) 1 clip = 30 hrtz, **
    if ((j < 60) and ((actvalue == data.iloc[ind,4]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        actvalue = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        activity3 = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 60):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,4]
    subvalue = data.iloc[ind,10]

dfal1 = pd.DataFrame(d)
dfal1.head()

In [ ]:
pickling_on = open("dfal1.pickle","wb") #activity 1 
pickle.dump(dfal1, pickling_on)
pickling_on.close()

### Splitting two second window for activity level 2

In [ ]:
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 60) and ((actvalue == data.iloc[ind,5]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        actvalue = data.iloc[ind,5]
        activity3 = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 60):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,5]
    subvalue = data.iloc[ind,10]

dfal2 = pd.DataFrame(d)
dfal2.head()

In [ ]:
pickling_on = open("dfal2.pickle","wb") #activity 2
pickle.dump(dfal2, pickling_on)
pickling_on.close()

### Splitting two second window for activity level 3

In [18]:
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 60) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 60):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal3 = pd.DataFrame(d)
dfal3.head()

,0,1,2
0,114,999,"[[0.879, -0.242, 0.406], [0.879, -0.242, 0.438..."
1,114,999,"[[0.883, -0.301, 0.391], [0.883, -0.297, 0.391..."
2,114,999,"[[-3.563, 0.004, -0.367], [-3.395, -0.547, 0.4..."
3,114,999,"[[-2.852, -0.629, 0.852], [-0.285, 0.785, -0.1..."
4,114,999,"[[-2.586, 0.508, -0.066], [-3.016, -0.211, -0...."


In [19]:
pickling_on = open("dfal3.pickle","wb") #activity 3 
pickle.dump(dfal3, pickling_on)
pickling_on.close()




In [ ]:
import pickle
pickle_off = open("dfal1.pickle","rb") #run 3 times for every activity (dfal # .pickle)
dfal1 = pickle.load(pickle_off)
print(dfal1.shape)

In [ ]:
dfal1.head()

In [ ]:
import itertools
import numpy as np
from io import StringIO
import math
import scipy.stats.stats as stats
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
def fft_positive(fft):
    half = math.floor(len(fft)/2)
    return fft[1:half]

In [ ]:
def weighted_fft_mean(signal):
    fft = abs(np.fft.fft(signal))
    t = np.arange(len(signal))
    fft_freq = np.fft.fftfreq(t.shape[-1])
    
    half = math.floor(len(fft)/2)    
    fft = fft[1:half]
    fft_freq = fft_freq[1:half]
    
    mean = np.sum(fft * fft_freq) / sum(fft_freq)
    
    return mean

In [ ]:
def fft_bin(axis,signal,bins):   
    bin_size = math.floor(len(signal)/bins)

    fft_features = [axis + ' fft bin ' + str(i+1) for i in range(bins)]    
#     fft_bins_mean = pd.DataFrame(index=[0], columns=fft_features)
    
    fft_bins = [signal[i:i + bin_size] for i in range(0, len(signal), bin_size)]
    fft_bins_mean = [np.mean(fft_bins[i]) for i in range(bins)]
    
#     for i in range(0,bins):
#         bin_mean_curr = np.mean(fft_bins[i]) 
#         fft_bins_mean.set_value(0, fft_features[i], bin_mean_curr)        
    
    return dict(zip(fft_features, fft_bins_mean))

In [ ]:
dfnew = dfal1
#where did you get the x y z cordinate values from?
dfnew['x'] = dfnew.apply(lambda row:  [i[0] for i in row[2]], axis = 1)
dfnew['y'] = dfnew.apply(lambda row:  [i[1] for i in row[2]], axis = 1)
dfnew['z'] = dfnew.apply(lambda row:  [i[2] for i in row[2]], axis = 1)
dfnew['magnitude'] = dfnew.apply(lambda row: [math.sqrt(i[0]**2 + i[1]**2 + i[2]**2) for i in row[2]], axis = 1)
dfnew['max'] = dfnew.apply(lambda row:  max(row['magnitude']), axis = 1)
dfnew['min'] = dfnew.apply(lambda row:  min(row['magnitude']), axis = 1)
dfnew['std'] = dfnew.apply(lambda row:  np.std(row['magnitude']), axis = 1)
dfnew['skew'] = dfnew.apply(lambda row:  stats.skew(row['magnitude']), axis = 1)
dfnew['kurtosis'] = dfnew.apply(lambda row:  stats.kurtosis(row['magnitude']), axis = 1)
dfnew['mean'] = dfnew.apply(lambda row:  np.mean(row['magnitude']), axis = 1)
dfnew['median'] = dfnew.apply(lambda row:  np.median(row['magnitude']), axis = 1)
dfnew['x max'] = dfnew.apply(lambda row:  max(row['x']), axis = 1)
dfnew['y max'] = dfnew.apply(lambda row:  max(row['y']), axis = 1)
dfnew['z max'] = dfnew.apply(lambda row:  max(row['z']), axis = 1)
dfnew['x min'] = dfnew.apply(lambda row:  min(row['x']), axis = 1)
dfnew['y min'] = dfnew.apply(lambda row:  min(row['y']), axis = 1)
dfnew['z min'] = dfnew.apply(lambda row:  min(row['z']), axis = 1)
dfnew['x mean'] = dfnew.apply(lambda row:  np.mean(row['x']), axis = 1)
dfnew['y mean'] = dfnew.apply(lambda row:  np.mean(row['y']), axis = 1)
dfnew['z mean'] = dfnew.apply(lambda row:  np.mean(row['z']), axis = 1)
dfnew['x median'] = dfnew.apply(lambda row:  np.median(row['x']), axis = 1)
dfnew['y median'] = dfnew.apply(lambda row:  np.median(row['y']), axis = 1)
dfnew['z median'] = dfnew.apply(lambda row:  np.median(row['z']), axis = 1)
dfnew['x skew'] = dfnew.apply(lambda row:  stats.skew(row['x']), axis = 1)
dfnew['y skew'] = dfnew.apply(lambda row:  stats.skew(row['y']), axis = 1)
dfnew['z skew'] = dfnew.apply(lambda row:  stats.skew(row['z']), axis = 1)
dfnew['x kurtosis'] = dfnew.apply(lambda row:  stats.kurtosis(row['x']), axis = 1)
dfnew['y kurtosis'] = dfnew.apply(lambda row:  stats.kurtosis(row['y']), axis = 1)
dfnew['z kurtosis'] = dfnew.apply(lambda row:  stats.kurtosis(row['z']), axis = 1)
dfnew['x std'] = dfnew.apply(lambda row:  np.std(row['x']), axis = 1)
dfnew['y std'] = dfnew.apply(lambda row:  np.std(row['y']), axis = 1)
dfnew['z std'] = dfnew.apply(lambda row:  np.std(row['z']), axis = 1)
dfnew['xy mean'] = dfnew.apply(lambda row:  np.mean(np.array(row['x']) * np.array(row['y'])), axis = 1)
dfnew['yz mean'] = dfnew.apply(lambda row:  np.mean(np.array(row['y']) * np.array(row['z'])), axis = 1)
dfnew['xz mean'] = dfnew.apply(lambda row:  np.mean(np.array(row['x']) * np.array(row['z'])), axis = 1)
temp = []
tempx = []
tempy = []
tempz = []
xrun = []
yrun = []
zrun = []
for i in range(len(dfnew)):
    temp = dfnew.iloc[i,2]
    
    for j in range(len(temp)):
        tempx.append(temp[j][0])
        tempy.append(temp[j][1])
        tempz.append(temp[j][2])
    x = np.array(tempx)
    y = np.array(tempy)
    z = np.array(tempz)
    
    xrun.append(movingaverage(x,20))
    yrun.append(movingaverage(y,20))
    zrun.append(movingaverage(z,20))
    tempx = []
    tempy = []
    tempz = []

dfnew.insert(7, "running x", xrun)
dfnew.insert(8, "running y", yrun)
dfnew.insert(9, "running z", zrun)
x = dfnew['x'] - dfnew['running x']
y = dfnew['y'] - dfnew['running y']
z = dfnew['z'] - dfnew['running z']

dfnew.insert(13, "x noise", x)
dfnew.insert(14, "y noise", y)
dfnew.insert(15, "z noise", z)
dfnew['x noise min'] = dfnew.apply(lambda row: min(row['x noise']), axis=1)
dfnew['y noise min'] = dfnew.apply(lambda row: min(row['y noise']), axis=1) 
dfnew['z noise min'] = dfnew.apply(lambda row: min(row['z noise']), axis=1)
dfnew['x noise max'] = dfnew.apply(lambda row: max(row['x noise']), axis=1) 
dfnew['y noise max'] = dfnew.apply(lambda row: max(row['y noise']), axis=1) 
dfnew['z noise max'] = dfnew.apply(lambda row: max(row['z noise']), axis=1)
dfnew['x noise std'] = dfnew.apply(lambda row: np.std(row['x noise']), axis=1) 
dfnew['y noise std'] = dfnew.apply(lambda row: np.std(row['y noise']), axis=1) 
dfnew['z noise std'] = dfnew.apply(lambda row: np.std(row['z noise']), axis=1)

#frequenny domain

x_fft = dfnew.apply(lambda row: [i for i in fft_positive(abs(np.fft.fft(row['x'])))], axis = 1)
y_fft = dfnew.apply(lambda row: [i for i in fft_positive(abs(np.fft.fft(row['y'])))], axis = 1)
z_fft = dfnew.apply(lambda row: [i for i in fft_positive(abs(np.fft.fft(row['z'])))], axis = 1)

dfnew.insert(10, 'x fft', x_fft)
dfnew.insert(11, 'y fft', y_fft)
dfnew.insert(12, 'z fft', z_fft)
dfnew['x fft min'] = dfnew.apply(lambda row: min(row['x fft']), axis = 1)
dfnew['y fft min'] = dfnew.apply(lambda row: min(row['y fft']), axis = 1)
dfnew['z fft min'] = dfnew.apply(lambda row: min(row['z fft']), axis = 1)
dfnew['x fft max'] = dfnew.apply(lambda row: max(row['x fft']), axis = 1)
dfnew['y fft max'] = dfnew.apply(lambda row: max(row['y fft']), axis = 1)
dfnew['z fft max'] = dfnew.apply(lambda row: max(row['z fft']), axis = 1)
dfnew['x fft std'] = dfnew.apply(lambda row: np.std(row['x fft']), axis = 1)
dfnew['y fft std'] = dfnew.apply(lambda row: np.std(row['y fft']), axis = 1)
dfnew['z fft std'] = dfnew.apply(lambda row: np.std(row['z fft']), axis = 1)
dfnew['x fft mean'] = dfnew.apply(lambda row: np.mean(row['x fft']), axis = 1)
dfnew['y fft mean'] = dfnew.apply(lambda row: np.mean(row['y fft']), axis = 1)
dfnew['z fft mean'] = dfnew.apply(lambda row: np.mean(row['z fft']), axis = 1)
dfnew['x fft median'] = dfnew.apply(lambda row: np.median(row['x fft']), axis = 1)
dfnew['y fft median'] = dfnew.apply(lambda row: np.median(row['y fft']), axis = 1)
dfnew['z fft median'] = dfnew.apply(lambda row: np.median(row['z fft']), axis = 1)
dfnew['x fft mean weighted'] = dfnew.apply(lambda row: weighted_fft_mean(row['x']), axis = 1)
dfnew['y fft mean weighted'] = dfnew.apply(lambda row: weighted_fft_mean(row['y']), axis = 1)
dfnew['z fft mean weighted'] = dfnew.apply(lambda row: weighted_fft_mean(row['z']), axis = 1)
dfnew = dfnew.merge(dfnew.x.apply(lambda row: pd.Series(fft_bin('x',row,10))), left_index=True, right_index=True)
dfnew = dfnew.merge(dfnew.y.apply(lambda row: pd.Series(fft_bin('y',row,10))), left_index=True, right_index=True)
dfnew = dfnew.merge(dfnew.z.apply(lambda row: pd.Series(fft_bin('z',row,10))), left_index=True, right_index=True)


In [ ]:
dfnew.head()

In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import metrics, svm, neighbors, linear_model, tree
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
%matplotlib inline

In [ ]:
df = dfnew

In [ ]:
df.head()

In [ ]:
df.rename(columns = {0:'subject', 1:'activity', 2:'raw acc data'},inplace=True)

In [ ]:
df.head()

In [ ]:
pickling_on = open("dffinal.pickle","wb") 
pickle.dump(df, pickling_on)
pickling_on.close()


In [ ]:
df.subject.unique() #participant number

In [ ]:
Y = df['activity']

In [ ]:
features = ['max', 'min', 'std', 'skew', 'kurtosis', 'mean', 'median', 'x max', 'y max', 'z max', 'x min', 'y min', 'z min', 'x mean', 'y mean', 'z mean', 'x median', 'y median', 'z median', 'x skew', 'y skew', 'z skew', 'x kurtosis', 'y kurtosis', 'z kurtosis', 'x std', 'y std', 'z std', 'xy mean', 'yz mean', 'xz mean','x fft min', 'y fft min', 'z fft min', 'x fft max', 'y fft max', 'z fft max', 'x fft std', 'y fft std', 'z fft std', 'x fft mean', 'y fft mean', 'z fft mean', 'x fft median', 'y fft median', 'z fft median', 'x fft mean weighted', 'y fft mean weighted', 'z fft mean weighted', 'x fft bin 1', 'x fft bin 10', 'x fft bin 2', 'x fft bin 3', 'x fft bin 4', 'x fft bin 5', 'x fft bin 6', 'x fft bin 7', 'x fft bin 8', 'x fft bin 9', 'y fft bin 1', 'y fft bin 10', 'y fft bin 2', 'y fft bin 3', 'y fft bin 4', 'y fft bin 5', 'y fft bin 6', 'y fft bin 7', 'y fft bin 8', 'y fft bin 9', 'z fft bin 1', 'z fft bin 10', 'z fft bin 2', 'z fft bin 3', 'z fft bin 4', 'z fft bin 5', 'z fft bin 6', 'z fft bin 7', 'z fft bin 8', 'z fft bin 9','x noise min','y noise min','z noise min','x noise max','y noise max','z noise max','x noise std','y noise std','z noise std']
X = df[features] #all features needed
feature_names = X.columns.tolist()
print(len(feature_names))
print(feature_names)
groups = df.subject.unique()
X.head()

In [ ]:
# features analysis
def select_features(X, Y, plot=False):
    forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
    
    feature_names = X.columns.tolist()

#     X = data[feature_names]
#     Y = data['activity']

    forest.fit(X, Y)
    importances = forest.feature_importances_ #using random forest to rank the features (features importance)
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]
    
    importances = np.array(importances)
    importances /= importances.max()

    if plot:
        for f in range(X.shape[1]):
            print("%d. feature '%s' (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))


        # Plot the feature importances of the forest
        plt.figure()
        plt.title("Feature importances")
        plt.bar(range(X.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
        plt.xticks(range(X.shape[1]), indices)
        plt.xlim([-1, X.shape[1]])
        plt.show()
        
    # select features > 25 % importance
    features = [i for i,j in zip(feature_names,importances) if j > 0.25]

    print ("Selected number of features: %d" % len(features))

    if plot:
        for f in range(len(features)):
            print("%d. feature %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))
        
    return features

In [ ]:
import matplotlib as mpl

mpl.rcParams["font.size"] = 24

class_names = ['Active', 'Inactive']
print(class_names)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

   
    print('Confusion matrix')

    

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# grid search with cross-validation (subject-wise or k-folds) for picking hyperparameters
# returns list with best estimators
# params:
# X_train - training data
# y_train - labels
# kfoldcv - true to use 10-fold cross-validation
# display - boolean for printing out cross-validation info
def grid(X_train,y_train, display=False, kfoldcv=True):
    # Different models to try
    #       Model name ---------------------------------------------------------------------
    #      Parameters --------------------
    ----------------------                           |
    #     Classifier -----------                               |                           |
    #                          |                               |                           |
    #                          v                               v                           |
    models = [[tree.DecisionTreeClassifier(), {'min_samples_split': [2, 4, 6, 8, 10],#    |
                                               'min_samples_leaf': [1, 5, 10, 15, 20],#     v
                                               'max_depth': [10, 20, 30, 40, 50]},       "Decision Tree"] # ?? models? is a list of algorithms?
              ]

    models.append([linear_model.LogisticRegression(), {'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Ridge Penalty"])

#     models.append([linear_model.LogisticRegression(penalty='l1'), {'C': [100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]}, "Logistic Regression with Lasso Penalty"])

    # LinearSVC
    models.append([svm.SVC(kernel='rbf'), { # class_weight="balanced"; tol 
                              'gamma': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6], 
                               'C': [1e+4, 1e+3, 1e+2, 1e+1, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]}, "SVM rbf"])
    
    models.append([neighbors.KNeighborsClassifier(), {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15]}, "K-Nearest Neighbors"])


    models.append([RandomForestClassifier(), {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400]
                                               }, "Random Forest"])

    models_with_best_params = [] 

    # cross-validation strategy
    # Leave One Group Out ??
    groups = y_train.unique()
    logo = LeaveOneGroupOut()
    cv = logo.get_n_splits(X_train,y_train,groups)
    
    # for 10-folds cross-validation
    if kfoldcv: 
        cv = 10 

    for model in models:
        clf = GridSearchCV(model[0], model[1], cv = cv) #http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
        clf.fit(X_train, y_train)
        best_params = clf.best_params_
        best_estimator = clf.best_estimator_

        model_with_best_params = [best_estimator, best_params, model[2]]

        models_with_best_params.append(model_with_best_params)

        if (display):
            print(model[2], ": ")
            print("Best score for ", model[2], ":", clf.best_score_)
            print()
            print("Best parameters for ", model[2], " found on development set:", best_params)
    #         print()
    #         print("Best estimator for ", model[2], " model:", best_estimator)
            print()
    #         print()

    #         print("Grid scores on development set:")
    #         print()
    #         for params, mean_score, scores in clf.grid_scores_:
    #             print("%0.3f (+/-%0.03f) for %r"
    #                   % (mean_score, scores.std() * 2, params))
            print()

    #     print(models_with_best_params)
    
#     return best_params['n_estimators']
    return  models_with_best_params

In [ ]:

def test_classifier (model, X_train, y_train, X_test, y_test):
    test_classifiers = []
#     classifier = RandomForestClassifier(n_estimators=pram)
    classifier = model[0]
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    #print("Prediction accuracy for", model[2], "model is", score)
    expected = y_test
    predicted = classifier.predict(X_test)

    print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted, digits=5)))
    #Compute confusion matrix
    cnf_matrix = confusion_matrix(expected, predicted)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plt.figure(figsize=(9,9))
    plot_confusion_matrix(cnf_matrix, classes=class_names,
    title='')

    plt.show()
    test_classifiers.append([metrics.f1_score(expected,predicted, average='weighted'), cnf_matrix])

    return [metrics.f1_score(expected,predicted, average='weighted'), cnf_matrix]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42) #split 7:3
print("Train Set Size: ", len(X_train))
print("Test Set Size: ", len(X_test))

features = select_features(X_train, y_train)
X_train = X_train[features]
print("Train set size after feature selection: ", X_train.shape)
X_test = X_test[features]
print("Test set size after feature selection: ", X_test.shape)

In [ ]:
best_classifiers = grid(X_train, y_train, True, True)

In [ ]:
#fitting models to test_data
for model in best_classifiers:
    test_classifier (model, X_train, y_train, X_test, y_test)